In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import h5py, io
import os
import torch
from torch.utils.data import Dataset,DataLoader,Sampler,Subset
from PIL import Image
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.metrics import roc_auc_score
import albumentations as A
from tqdm.notebook import tqdm_notebook as tqdm
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
import torchvision
import torchmetrics 
from torchvision import datasets, models, transforms
import sklearn
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_auc_score
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import warnings
import pickle
warnings.filterwarnings("ignore")
sns.set()

In [2]:
train_metadata = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')

In [3]:
image_data = h5py.File('/kaggle/input/isic-2024-challenge/train-image.hdf5' , 'r')

In [4]:
train_grp = list(image_data.keys())

In [5]:
groups = train_metadata['patient_id']

In [6]:
y = train_metadata['target']

In [9]:
sampling_ratio = 0.01
sampler = RandomUnderSampler(sampling_strategy = sampling_ratio, random_state = 42)
sampled_df, y_sample = sampler.fit_resample(df , y)

In [10]:
# train_keys, val_keys, y_train, y_val = train_test_split(train_grp, train_metadata.target, shuffle = True, train_size = 0.9 , random_state = 42, stratify = train_metadata.target)

In [11]:
sampled_keys = sampled_df['keys']
groups = train_metadata['patient_id']

In [7]:
class dataset(Dataset):
    def __init__(self, transform , image_data, keys, y, mode):
        super().__init__()
        self.mode = mode
        self. y = y
        self.keys = keys
        self.image_data = image_data
        self.transform = transform
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        image = np.array(self.image_data[self.keys[idx]]) 
        image = np.array(Image.open(io.BytesIO(image)),dtype=np.float32)/255
        
        augmented = self.transform(image = image) 
        image = augmented['image'] 
        
        if self.mode == 'test':
            return image
        
        label = int(self.y[idx])

        return image, label  

In [8]:
train_transform = A.Compose([
    A.Resize(height=224, width=224),
    # A.OneOf([A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15),
    #          A.RandomBrightnessContrast() 
    #          ], p=0.5),
    # A.HorizontalFlip(p=0.5),
    # A.VerticalFlip(p=0.5),
    A.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=1.0),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(height = 224, width = 224),
    A.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=1.0),
    ToTensorV2(),
])

In [9]:
# train_transform =  A.Compose([
#         # A.Resize(CONFIG['img_size'], CONFIG['img_size']),
#     A.Resize(height=224, width=224),
#         A.RandomRotate90(p=0.5),
#         A.Flip(p=0.5),
#         A.Downscale(p=0.25),
#         A.ShiftScaleRotate(shift_limit=0.1, 
#                            scale_limit=0.15, 
#                            rotate_limit=60, 
#                            p=0.5),
#         A.HueSaturationValue(
#                 hue_shift_limit=0.2, 
#                 sat_shift_limit=0.2, 
#                 val_shift_limit=0.2, 
#                 p=0.5
#             ),
#         A.RandomBrightnessContrast(
#                 brightness_limit=(-0.1,0.1), 
#                 contrast_limit=(-0.1, 0.1), 
#                 p=0.5
#             ),
# #         A.Normalize(
# #                 mean=[0.485, 0.456, 0.406], 
# #                 std=[0.229, 0.224, 0.225], 
# #                 max_pixel_value=255.0, 
# #                 p=1.0
# #             ),
#         ToTensorV2()], p=1)
    
# val_transform =  A.Compose([
#            A.Resize(height=224, width=224),
# #         A.Normalize(
# #                 mean=[0.485, 0.456, 0.406], 
# #                 std=[0.229, 0.224, 0.225], 
# #                 max_pixel_value=255.0, 
# #                 p=1.0
# #             ),
#         ToTensorV2()])

In [10]:
y_train = torch.tensor(np.array(y_train))
y_val = torch.tensor(np.array(y_val))

In [11]:
train_dataset = dataset(train_transform, image_data, np.array(train_grp) , np.array(y) , mode = 'train')
# val_dataset = dataset(val_transform, image_data, val_keys, y_val, mode = 'train')

In [12]:
class StratifiedBatchSampler:
    def __init__(self, y, batch_size, shuffle=True):
        if torch.is_tensor(y):
            y = y.numpy()
        assert len(y.shape) == 1, 'label array must be 1D'
        n_batches = int(len(y) / batch_size)
        self.skf = StratifiedKFold(n_splits=n_batches, shuffle=shuffle)
        self.X = torch.randn(len(y),1).numpy()
        self.y = y
        self.shuffle = shuffle

    def __iter__(self):
        if self.shuffle:
            self.skf.random_state = torch.randint(0,int(1e8),size=()).item()
        for train_idx, test_idx in self.skf.split(self.X, self.y):
            yield test_idx

    def __len__(self):
        return len(self.y)

In [13]:
def make_weights_for_balanced_classes(y, nclasses):
    n_images = len(y)
    count_per_class = [0] * nclasses
    for i in y:
        count_per_class[i] += 1
    weight_per_class = [0.] * nclasses
    for i in range(nclasses):
        weight_per_class[i] = float(n_images) / float(count_per_class[i])
    weights = [0] * n_images
    for idx, label in enumerate(y):
        weights[idx] = weight_per_class[label]
    return weights

In [291]:
# weights = make_weights_for_balanced_classes(y_train,2)                                                                
# weights = torch.DoubleTensor(weights)                                       
# sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))                     
                                                                                
train_dataloader = DataLoader(train_dataset,
                           batch_size = 32,      
                           shuffle = True,
                           # sampler = sampler, 
                           num_workers = 0, 
                           pin_memory = False)  

# train_dataloader = DataLoader(train_dataset,
#                              # sampler = StratifiedBatchSampler(y_train, 32, shuffle = True),
#                               # sampler = sampler,
#                               # sampler = torch.utils.data.sampler.SequentialSampler(train_dataset),
#                               batch_size = 32, 
#                               shuffle = True,
#                              num_workers = 0,
#                              pin_memory = False)

# val_dataloader = DataLoader(val_dataset,
#                            batch_size = 32,
#                            shuffle = False,
#                            num_workers = 0,
#                            pin_memory = False)

In [54]:
len(train_dataloader)

1023

In [55]:
im , lb =  next(iter(train_dataloader))

In [14]:
def score(y_true, y_pred):
    with torch.inference_mode():
        y_pred = nn.functional.sigmoid(y_pred)
    y_true = y_true.cpu()
    y_pred = y_pred.detach().cpu()
    min_tpr=0.8
    v_gt = abs(np.asarray(y_true)-1)
    v_pred = np.array([1.0 - x for x in y_pred])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc

In [15]:
def roc_score(y_true, y_pred):
    with torch.inference_mode():
        y_pred = nn.functional.sigmoid(y_pred)
    y_true = y_true.cpu()
    y_pred = y_pred.detach().cpu()
    return roc_auc_score(y_true, y_pred)

In [24]:
# from torchvision.models import resnet18, ResNet18_Weights
# model = models.resnet18(weights = ResNet18_Weights.IMAGENET1K_V1).to(device)

In [16]:
import timm
model = timm.create_model("hf_hub:timm/densenet121.tv_in1k", pretrained=True)

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/32.3M [00:00<?, ?B/s]

In [ ]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)

    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'

In [17]:
for param in model.parameters():
    param.requires_grad = False   

In [18]:
model.classifier

Linear(in_features=1024, out_features=1000, bias=True)

In [19]:
model.classifier = nn.Sequential(
#                 nn.Flatten(start_dim = 1, end_dim = -1)
                nn.Linear(1024 , 1),
                # nn.ReLU(inplace = True),
                # nn.Dropout(p = 0.5, inplace = False),
                # nn.Linear(256,128),
                # nn.ReLU(),
                # nn.Linear(128,1)
#                 nn.Sigmoid()
).to(device)
model.classifier.requires_grad = True

In [20]:
model = model.to(device)

In [21]:
n_epochs = 2
# loss = torchvision.ops.sigmoid_focal_loss
loss = nn.BCEWithLogitsLoss()
# optimizer = torch.optim.Adam(model.classifier.parameters())
optimizer = torch.optim.AdamW(model.classifier.parameters(), lr = 1e-3)
# LR_SCHEDULER = torch.optim.lr_scheduler.StepLR(optimizer,5,0.1)
cos_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 500, eta_min = 1e-6)

In [22]:
next(model.parameters()).is_cuda

True

In [23]:
def stratified_group_k_fold(data, labels, groups, n_splits=3, random_state=42):
    skf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for train_idx, val_idx in skf.split(data, labels, groups):
        yield train_idx, val_idx

In [24]:
def StratifiedGroupSplitKFold(X, y, groups, n_splits=5, test_size=0.2, random_state=None):
    gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    
    for _ in range(n_splits):
        # Get train and test indices using GroupShuffleSplit
        train_idx, test_idx = next(gss.split(X, y, groups))
        
        # Ensure train_idx and test_idx are NumPy arrays
        train_idx = np.array(train_idx)
        test_idx = np.array(test_idx)

        # Extract training and test sets based on indices
        X_train = np.array([X[i] for i in train_idx])
        X_test = np.array([X[i] for i in test_idx])
        y_train = np.array([y[i] for i in train_idx])
        y_test = np.array([X[i] for i in test_idx])
        
        # X_train, X_test = X[train_idx], X[test_idx]
        # y_train, y_test = y[train_idx], y[test_idx]
        
        # Stratify the training set using StratifiedKFold
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
        
        for inner_train_idx, inner_val_idx in skf.split(X_train, y_train):
            # Ensure inner_train_idx and inner_val_idx are NumPy arrays
            inner_train_idx = np.array(inner_train_idx)
            inner_val_idx = np.array(inner_val_idx)
            
            # Map back to original indices
            # final_train_idx = train_idx[inner_train_idx]
            # final_val_idx = train_idx[inner_val_idx]

            final_train_idx = [train_idx[i] for i in inner_train_idx]
            final_val_idx = [train_idx[i] for i in inner_val_idx]

            yield (final_train_idx, final_val_idx)

In [25]:
n_splits = 2
kfold = StratifiedGroupSplitKFold(train_grp, y, np.array(groups), n_splits=n_splits, test_size=0.1, random_state=42)

In [ ]:
# for fold, (train_idx, val_idx) in enumerate(stratified_group_k_fold(np.array(train_grp) , np.array(y) , np.array(train_metadata['patient_id']))):
#     train_subset = Subset(train_dataset, train_idx)
#     val_subset = Subset(train_dataset, val_idx)

for fold, (train_idx, val_idx) in enumerate(kfold):
    
    train_subset = Subset(train_dataset, train_idx)
    val_subset = Subset(train_dataset, val_idx)
    
    train_dataloader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers = 0)
    val_dataloader = DataLoader(val_subset, batch_size=32, shuffle=False, num_workers = 0)
    for epoch in range(n_epochs):
        print('Epoch {}/{}'.format(epoch+1, n_epochs))
        print('-' * 10)
        model.train()
        torch.cuda.empty_cache()
        net_loss = 0.0
        train_pauc = 0.0
        train_preds = []
        train_labels = []
        val_preds = []
        val_labels = []
        for image, labels in tqdm(train_dataloader):
            image = image.to(device)
            labels = labels.reshape(-1,1).to(torch.float32).to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                pred = model(image)
                iter_loss = loss(pred, labels)
                net_loss+=(iter_loss.item())/len(train_dataloader)
                iter_loss.backward()
                optimizer.step()
            with torch.inference_mode():
                train_preds.append(pred)
                train_labels.append(labels)   
                del image, labels, pred
        with torch.inference_mode():
            val_pauc = 0.0
            val_preds = []
            val_labels = []
            for image, labels in val_dataloader:
                image = image.to(device)
                labels = labels.reshape(-1,1).to(torch.float32).to(device)
                pred = model(image)
                val_labels.append(labels)
                val_preds.append(pred)
                del image, labels, pred
            train_pauc = score(torch.cat(train_labels, dim = 0).reshape(-1,1), torch.cat(train_preds, dim = 0).reshape(-1,1))
            val_pauc = score(torch.cat(val_labels, dim = 0).reshape(-1,1), torch.cat(val_preds, dim = 0).reshape(-1,1))
        print(f"Epoch = {epoch+1} | train_loss: {net_loss} | train_pauc : {train_pauc} | val_pauc : {val_pauc}")
        gc.collect()
        LR_SCHEDULER.step()

Epoch 1/2
----------


  0%|          | 0/5613 [00:00<?, ?it/s]

Epoch = 1 | train_loss: 0.00857085975301674 | train_pauc : 0.040945860309086646 | val_pauc : 0.08124110800806203
Epoch 2/2
----------


  0%|          | 0/5613 [00:00<?, ?it/s]

Epoch = 2 | train_loss: 0.00730040310885066 | train_pauc : 0.07245552420699136 | val_pauc : 0.07509277184922017
Epoch 1/2
----------


  0%|          | 0/5613 [00:00<?, ?it/s]

Epoch = 1 | train_loss: 0.007510838265047192 | train_pauc : 0.059913636211636485 | val_pauc : 0.0946066097900071
Epoch 2/2
----------


  0%|          | 0/5613 [00:00<?, ?it/s]

In [75]:
train_pauc = score(torch.cat(train_labels, dim = 0).reshape(-1,1), torch.cat(train_preds, dim = 0).reshape(-1,1))
train_pauc

0.09724794329745355

In [ ]:
embeddings = []

In [109]:
with torch.inference_mode():
    for image, _ in train_dataloader:
        image = image.to(device)
        embeddings.append(model(image).cpu().numpy())
        del image, _

In [56]:
test_preds = []
test_labels = []
with torch.inference_mode():
    for img,labels in tqdm(train_dataloader):
        img = img.to(device)
        test_preds.append(model(img))
        test_labels.append(labels)

  0%|          | 0/11280 [00:00<?, ?it/s]

In [57]:
temp_preds = torch.cat(test_preds, dim = 0).cpu()
temp_y = torch.cat(test_labels, dim = 0).cpu()

In [325]:
from sklearn.metrics import roc_auc_score
roc_auc_score(temp_y, temp_preds)

In [ ]:
pickle.dump(model, open("resnet18_model14.sav", 'wb'))

In [111]:
pickle.dump(embeddings, open('resnet18_embeddings_unshuffled', 'wb'))